# EDA Suzano - Análise Exploratória de Dados

## Análise de Mercado: Suzano, Celulose, Câmbio e Fatores Macroeconômicos

---

### Objetivos

Este notebook apresenta uma análise exploratória completa relacionando:

1. **Preço das ações SUZB3** (Suzano S.A.)
2. **Preço da celulose** (BHKP/NBSK em USD)
3. **Taxa de câmbio PTAX** (USD/BRL)
4. **Taxa SELIC** (juros brasileiros)
5. **Fatores climáticos** (precipitação e NDVI)
6. **Índice de crédito**

### Metodologia

- **Frequência**: Dias úteis (Business days) com forward fill para dados mensais
- **Transformações**: Log-retornos para todas as séries
- **Defasagens**: Lags climáticos de 15, 30 e 60 dias
- **Índice Sintético**: Regressão OLS com z-scores para sinais de trading
- **Cointegração**: Teste de Johansen e modelo VECM

---


## 1. Setup e Imports


In [ ]:
# Import libraries
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import EDA modules
from eda.config import DATA_INT, DATA_OUT, PLOTS_DIR
from eda.features import build_features
from eda.synthetic import fit_synthetic, analyze_coefficients, backtest_signals
from eda.models import fit_vecm, run_adf_tests
from eda.plots import generate_all_plots

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports successful!")


---

## 2. Carregamento e Preparação dos Dados

### 2.1 Ingestão de Dados

Carregamos dados de múltiplas fontes:
- **Yahoo Finance**: SUZB3.SA (ações Suzano)
- **BCB API**: PTAX e SELIC
- **CSV**: Preços da celulose, clima e crédito


In [ ]:
# Build feature matrix
df = build_features(start='2020-01-01')

print(f"\n📊 Shape: {df.shape}")
print(f"📅 Period: {df.index.min()} to {df.index.max()}")
print(f"📋 Columns: {df.columns.tolist()}")


### 2.2 Primeiras Observações e Estatísticas


In [ ]:
# Display first and last rows
print("First rows:")
display(df.head())
print("\nLast rows:")
display(df.tail())
print("\nDescriptive statistics:")
display(df.describe())


---

## 3. Análise Exploratória

### 3.1 Visualização de Séries Temporais


In [ ]:
# Plot key level variables
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

if 'suzb' in df.columns:
    df['suzb'].plot(ax=axes[0], title='SUZB3 - Preço das Ações (BRL)', color='steelblue')
    axes[0].set_ylabel('Preço (BRL)')
    axes[0].grid(True, alpha=0.3)

if 'pulp_brl' in df.columns:
    df['pulp_brl'].plot(ax=axes[1], title='Celulose - Preço em BRL', color='darkorange')
    axes[1].set_ylabel('Preço (BRL/ton)')
    axes[1].grid(True, alpha=0.3)

if 'ptax' in df.columns:
    df['ptax'].plot(ax=axes[2], title='PTAX - Taxa de Câmbio USD/BRL', color='green')
    axes[2].set_ylabel('BRL por USD')
    axes[2].grid(True, alpha=0.3)

if 'selic' in df.columns:
    df['selic'].plot(ax=axes[3], title='SELIC - Taxa de Juros (%)', color='red')
    axes[3].set_ylabel('Taxa (%)')
    axes[3].set_xlabel('Data')
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

## 4. Índice Sintético e Z-Scores

### 4.1 Regressão OLS

Construímos um índice sintético usando regressão OLS para modelar SUZB3 baseado em fatores macroeconômicos.


In [ ]:
# Fit synthetic index
model, synthetic, zscore, signals = fit_synthetic(df)

# Analyze coefficients
coef_df = analyze_coefficients(model)
print("\n=== Análise de Coeficientes ===")
display(coef_df)

print(f"\nR-squared: {model.rsquared:.4f}")
print(f"R-squared ajustado: {model.rsquared_adj:.4f}")


### 4.2 Visualização: Índice Sintético vs Real e Z-Scores


In [ ]:
# Plot actual vs synthetic with z-scores
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

aligned_idx = df.index.intersection(synthetic.index)
if 'suzb_r' in df.columns:
    df.loc[aligned_idx, 'suzb_r'].plot(ax=ax1, label='Real', color='steelblue', linewidth=1.5)
    synthetic.loc[aligned_idx].plot(ax=ax1, label='Sintético', color='darkorange', linewidth=1.5)
    ax1.set_title('SUZB_R: Real vs Índice Sintético')
    ax1.set_ylabel('Retorno')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

zscore.plot(ax=ax2, color='darkgreen', linewidth=1.5)
ax2.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax2.axhline(2, color='red', linestyle='--', linewidth=1, label='Threshold (+2)')
ax2.axhline(-2, color='red', linestyle='--', linewidth=1, label='Threshold (-2)')
ax2.fill_between(zscore.index, 2, zscore, where=(zscore > 2), color='red', alpha=0.3)
ax2.fill_between(zscore.index, -2, zscore, where=(zscore < -2), color='red', alpha=0.3)
ax2.set_title('Z-Score do Spread (Janela Rolling de 60 dias)')
ax2.set_ylabel('Z-Score')
ax2.set_xlabel('Data')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nNúmero de sinais de compra: {(signals == 1).sum()}")
print(f"Número de sinais de venda: {(signals == -1).sum()}")


### 4.3 Backtest Simples


In [ ]:
# Simple backtest
backtest = backtest_signals(df, signals, returns_col='suzb_r')

fig, ax = plt.subplots(figsize=(14, 6))
backtest['cumulative_market'].plot(ax=ax, label='Buy & Hold', color='steelblue', linewidth=2)
backtest['cumulative_strategy'].plot(ax=ax, label='Estratégia Z-Score', color='darkorange', linewidth=2)
ax.axhline(0, color='black', linestyle='--', linewidth=0.8)
ax.set_title('Backtest: Retornos Acumulados')
ax.set_ylabel('Retorno Acumulado')
ax.set_xlabel('Data')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

market_return = backtest['cumulative_market'].iloc[-1]
strategy_return = backtest['cumulative_strategy'].iloc[-1]

print(f"\nRetorno do mercado: {100*market_return:.2f}%")
print(f"Retorno da estratégia: {100*strategy_return:.2f}%")
print(f"Retorno excedente: {100*(strategy_return - market_return):.2f}%")


---

## 5. Cointegração e VECM

### 5.1 Testes de Estacionariedade (ADF)


In [ ]:
# ADF tests
test_cols = ['pulp_brl', 'suzb', 'pulp_brl_r', 'suzb_r']
adf_results = run_adf_tests(df, cols=test_cols)

print("\n=== Resultados dos Testes ADF ===")
display(adf_results[['column', 'adf_statistic', 'p_value', 'stationary']])


### 5.2 Teste de Johansen e Modelo VECM


In [ ]:
# Fit VECM
johansen_result, vecm_result = fit_vecm(df, cols=('pulp_brl', 'suzb'), k_ar_diff=2)

print("\n=== Vetores de Cointegração (Beta) ===")
print(vecm_result.beta)

print("\n=== Coeficientes de Ajuste (Alpha) ===")
print(vecm_result.alpha)

print("\n📊 Interpretação:")
print("- Beta: Define a relação de equilíbrio de longo prazo")
print("- Alpha: Velocidade de ajuste ao equilíbrio")
print("- Se cointegradas, desvios do equilíbrio são temporários")


---

## 6. Conclusões

### Principais Achados

1. **Correlações**: SUZB3 correlaciona-se com celulose, câmbio e juros
2. **Índice Sintético**: Captura variações de SUZB3 e identifica desvios
3. **Sinais de Trading**: Z-scores fornecem pontos de entrada/saída
4. **Cointegração**: Relação de longo prazo entre celulose e SUZB3

### Próximos Passos

1. Integrar dados reais de celulose (FOEX), clima (INMET) e crédito (BCB)
2. Backtesting robusto com custos de transação
3. Modelos não-lineares e machine learning
4. Dashboard interativo para monitoramento

---
